#Portfolio Optimizer using [PyPortfolioOpt](https://github.com/robertmartin8/PyPortfolioOpt/tree/master), from [Robert Martin](https://github.com/robertmartin8) </br> Data updates every weekday 6pm ET, [Data](https://github.com/carter4299/update_resources) </br>Notebook constructed by [Carter Susi](https://github.com/carter4299)

--------------------------------------------------------------------------------

##Press the play buttons in order





In [ ]:
#@title
!pip install PyPortfolioOpt yfinance pandas numpy

In [4]:
#@title
import yfinance as yf
import pandas as pd
from datetime import date
import numpy as np
from pypfopt import (
    CLA,
    BlackLittermanModel,
    EfficientFrontier,
    HRPOpt,
    black_litterman,
    expected_returns,
    plotting,
    risk_models,
)

--------------------------------------------------------------------------------

In [ ]:
#@title Enter your stocks and amounts
#@markdown Enter the stocks you own, separated by commas:
stocks = 'AAPL,GOOG,META,BP,TSLA' #@param {type:"string"}
#@markdown Enter the amount of each stock you own, in the same order, separated by commas:
amounts = '10.5,15.2,20.7,40.9,9.4' #@param {type:"string"}
try:
    stocks_list = [stock.strip() for stock in stocks.split(',')]
    amounts_list = [float(amount.strip()) for amount in amounts.split(',')]
    if len(stocks_list) != len(amounts_list):
        raise ValueError("The number of stocks and amounts do not match.")
    portfolio = dict(zip(stocks_list, amounts_list))
    init_portfolio, tp = {}, 0
    for stock in stocks_list:
        last_quote = yf.Ticker(stock).history()['Close'].iloc[-1]
        tp += last_quote * portfolio[stock]
        init_portfolio[stock] = f'$ {last_quote * portfolio[stock]}'
    print(f'Youre initial portfolio {init_portfolio}')
except ValueError as e:
    print("Error:", e)

In [ ]:
#@title Update SPY and Market Caps
def ret_stock_prices():
    m =  pd.read_csv('https://raw.githubusercontent.com/carter4299/custom-PPO-Notebook/main/data/tick_info.csv')
    sp = []
    mc = {}
    for stock in stocks_list:
        print(f'{stock} .......')
        y = (yf.download(stock, start="1900-01-01", end=str(date.today()), interval='1d'))
        sp.append(y.iloc[:, 4].rename(stock))
        mc[stock] = m.loc[m['s'] == stock, 'm'].values[0]

    return pd.concat(sp, axis=1), mc

def ret_spy_prices():
    print("SPY .......")
    y = (yf.download(stock, start="1900-01-01", end=str(date.today()), interval='1d'))
    return y.iloc[:, 4].rename(stock)

stock_prices, mcaps = ret_stock_prices()
spy_prices = ret_spy_prices()
assert stock_prices is not None or spy_prices is not None, "Import Failure"
print("Import Successful")


--------------------------------------------------------------------------------

In [ ]:
#@title Optimizer
returns = stock_prices.pct_change().dropna()
mu = expected_returns.mean_historical_return(stock_prices)
S = risk_models.sample_cov(stock_prices)


# Now try with a nonconvex objective from  Kolm et al (2014)
def deviation_risk_parity(w, cov_matrix):
    diff = w * np.dot(cov_matrix, w) - (w * np.dot(cov_matrix, w)).reshape(-1, 1)
    return (diff**2).sum().sum()


ef = EfficientFrontier(mu, S)
weightz = ef.nonconvex_objective(deviation_risk_parity, ef.cov_matrix)
print('Nonconvex Objective\n')
print({stock: f'$ {weightz[stock] * tp}' for stock in stock_prices})
ef.portfolio_performance(verbose=True)


# Hierarchical risk parity
hrp = HRPOpt(returns)
weights = hrp.optimize()
print("\nHierarchical risk parity")
print({stock: f'$ {weights[stock] * tp}' for stock in stock_prices})
hrp.portfolio_performance(verbose=True)
plotting.plot_dendrogram(hrp)  # to plot dendrogram



# Crticial Line Algorithm
cla = CLA(mu, S)
print('\nCrticial Line Algorithm')
clay = cla.max_sharpe()
print({stock: f'$ {clay[stock] * tp}' for stock in stock_prices})
cla.portfolio_performance(verbose=True)
print('\nEfficient Frontier Plot')
plotting.plot_efficient_frontier(cla)  # to plot